In [37]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/video/master/video_games_sale.csv", index_col=0)
df.head(3)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,Year_of_ten
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E,2000.0
1,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,2000.0
2,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E,2000.0


### video_games_sales 데이터셋(video_games_sales.csv)의 출시년도(Year_of_Release) 컬럼을 10년단위(ex 1990~1999 : 1990)로 변환하여 새로운 컬럼(year_of_ten)에 추가하고 게임이 가장 많이 출시된 년도(10년단위)와 가장 적게 출시된 년도(10년단위)를 각각 구하여라.

In [50]:
df['year_of_ten'] = df['Year_of_Release'].apply(lambda x: int(x/10)*10)
df['year_of_ten'].value_counts().index[0], df['year_of_ten'].value_counts().index[-1]

(2000, 1980)

### 플레이스테이션 플랫폼 시리즈(PS,PS2,PS3,PS4,PSV)중 장르가 Action로 발매된 게임의 총 수는?



In [56]:
sum(df[df['Platform'].isin(['PS','PS2','PS3','PS4','PSV'])]['Genre']=='Action')

616

### 게임이 400개 이상 출시된 플랫폼들을 추출하여 각 플랫폼의 User_Score 평균값을 구하여 데이터프레임을 만들고 값을 내림차순으로 정리하여 출력하라

In [103]:
tmp = df['Platform'].value_counts().reset_index()
li = list(tmp[tmp['Platform']>=400]['index'])
tmp = df[df['Platform'].isin(li)].groupby('Platform').mean()['User_Score']
# frame 방법 1
pd.DataFrame(tmp.sort_values(ascending=False))
# frame 방법 2
tmp.sort_values(ascending=False).to_frame()

,User_Score
Platform,
PS2,7.664386
XB,7.540708
PC,7.060369
DS,7.047198
Wii,6.903758
PS3,6.782705
X360,6.780769


### 게임 이름에 Mario가 들어가는 게임을 3회 개발한 개발자(Developer컬럼)을 구하여라



In [137]:
tmp = df['Name'].apply(lambda x: True if 'Mario' in x else False)
tmp = df.loc[tmp].groupby('Developer').count()['Name'].sort_values(ascending=False)
print(tmp.loc[tmp==3].index.values)

['Nintendo, Nintendo Software Technology' 'Nintendo, Nd Cube']


#### str 함수를 활용한 다른 답안

In [143]:
tmp = df[df.Name.str.contains('Mario')].Developer.value_counts()
print(tmp.loc[tmp==3].index.values)

['Nintendo, Nd Cube' 'Nintendo, Nintendo Software Technology']


### PS2 플랫폼으로 출시된 게임들의 User_Score의 첨도를 구하여라



In [147]:
df[df['Platform'] == 'PS2']['User_Score'].kurtosis()

2.4920035946875965

### 각 게임별 NA_Sales,EU_Sales,JP_Sales,Other_Sales 값의 합은 Global_Sales와 동일해야한다. 소숫점 2자리 이하의 생략으로 둘의 값의 다른경우가 존재하는데, 이러한 케이스가 몇개 있는지 확인하라



In [154]:
len(df) - sum(df['Global_Sales'] == df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales'])

3636

### User_Count컬럼의 값이 120 이상인 게임들 중에서 User_Score의 값이 9.0이상인 게임의 수를 구하여라



In [163]:
sum(df[df['User_Count'] >= 120]['User_Score']>=9.0)

86

### Global_Sales컬럼의 값들을 robust스케일을 진행하고 40이상인 데이터 수를 구하여라



In [208]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
tmp = scaler.fit_transform(df['Global_Sales'].values.reshape(-1,1))
len(tmp[tmp>=40])

6